In [5]:
import numpy as np
import pandas as pd #just in case i guess idk
import matplotlib.pyplot as plt #for graphing?
import math #this is how to get pi
import iop_ext as iop #artem's code for the radius/error functions - irradiated ocean planets

In [6]:
#DEFINE CONSTANTS
Lsun = 3.846*10**(26) #luminosity of sun in watts
Msun = 1.9891*10**(30) #mass of sun in kg
Mearth = 5.97219*10**(24) #mass of earth in kg
Rearth = 6.371*10**6 #radius of earth in m
au = 1.49598*10**11 #astronomical unit (distance from earth to sun) in m
yr = 3600*24*365 #converting years to seconds
a0 = 0.5 #slope at which Lhe decreases over time
tsat = 100*1e6*yr #in sec (length of the saturation phase?)
G = 6.67 * 10**(-11) #universal gravitational constant
pi = math.pi #pi

#PLANET: 
Lstar =  #luminosity of host star
Mstar =  #mass of host star
Teq =  #equillibrium temperature of planet in K
a = #semimajor axis
x0 = #initial WMF

Lsat = (10**(-3.5))*Lsun*(Mstar/Msun) #Lhe (high energy luminosity - Xrays and UV) during the saturation phase (Lhe remains constant)
CMFprime = 0.325 #planet reduced core mass fraction (cmf') - same as earth


#go to NEA, pick exoplanet and put all that data in here ^

SyntaxError: invalid syntax (1745727279.py, line 16)

In [7]:
#INITIALIZE VARIABLES (will be added to in the loop but this is where they start)
t = 1e6*yr #initial time
M0 = 5*Mearth #initial mass of planet
Mp = M0 #planet mass

NameError: name 'yr' is not defined

In [ ]:
#DEFINE FUNCTIONS
def RpFunction(CMF, x, Teq, Mp): #function to find the radius of the planet in m
    radius = iop.radius_iop(CMF, x, Teq, Mp/Mearth) #computes the radius of a planet, using the IOP model from Aguichine 2021
    return radius*Rearth

def nFunction(Mp, Rp): #n = efficiency parameter (greek letter eta) (unitless) - how much high energy is converted into mass loss
    vesc = ((2*G*Mp)/(Rp))**(1/2) #escape velocity at the planet's surface
    n = 0.1*((vesc/15)**-2)
    return n

def LheFunction(t): #Lhe = high energy luminosity of the host star
    if (t<tsat):
        return Lsat
    else:
        Lhe = Lsat*((t/tsat)**(-1-a0))
        return Lhe

def mFunction(Mp, Rp, t): #first time derivative of Mp (mass loss rate) in kg/s
    Lhe = LheFunction(t)
    n = nFunction(Mp, Rp)
    m = (n*(((pi)*(Rp**3)*(Lhe))/(4*(pi)*(a**2)*(G)*(Mp))))
    return m #this returns the mass loss rate in kilograms per second
#to get mass loss in earth masses per billion years, multiply result by 1e9*yr and divide by Mearth (see test code from meeting)

def xwmfFunction(Mp): #x = water mass fraction
    # original guess was x = (Mp-(M0*x0))/Mp but I fear that may be wrong
    x = (M0*x0-M0+Mf)/Mf
    #basically W0(initial amount of water) = M0*x0, Mlost = M0-Mf, Wf(final amount of water) = W0-Mlost, x = (Wf/Mf)
    return x

In [3]:
#TEST CODE FROM MEETING
x = xwmfFunction(Mp)
Rp = RpFunction(CMFprime, x, Teq, Mp)
print(RpFunction(CMFprime, x, Teq, Mp)/Rearth) #radius IN EARTH RADII
print(mFunction(Mp, Rp, t)*(1e9*yr)/Mearth) #this gives mass loss in earth masses per billion years

NameError: name 'xwmfFunction' is not defined

In [4]:
#EVOLUTION

deltat = 1e6*yr #incrimenting by 1 million years

#what about the errors function in iop_ext ???

while (t < (5*1e9*yr)): #after it has evolved for 5 billion years
    x = xwmfFunction(Mp) 
    Rp = RpFunction(CMFprime, x, Teq, Mp)
    m = mFunction(Mp, Rp, t)
    #incriment variables for next loop
    t = t+deltat
    Mp = Mp-(m*deltat)

NameError: name 'yr' is not defined